In [1]:
from utils import set_up_altair, moving_averages, read_json_to_df, format_time_columns,aggregate_by_year_month, filter_by_year
from statsmodels.tsa.seasonal import STL as STL
import pandas as pd
import altair as alt
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.tsa.stattools as ts
import nltk
import re
from nltk.tokenize import word_tokenize

PATH = "../../data/all_incidents.json"
data = read_json_to_df(PATH)
data = format_time_columns(data)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:

text_data = data.main_text[0]
text_data

'A short alert was quickly resolved after a walker was concerned for another walker and his two dogs who was the high side of Lingmell Gill which looked to be unpassable due to the heavy rainfall. However the walker was able to communicate his intentions to re-ascend to a higher elevation, cross the gill and take an alternative route down. With no assistance required, the log was closed.  '

In [9]:
cv = CountVectorizer(ngram_range=(2,2))

In [12]:
bigram = cv.fit_transform(data.main_text[0:5])

In [15]:
print(cv.vocabulary_)

{'short alert': 188, 'alert was': 5, 'was quickly': 255, 'quickly resolved': 170, 'resolved after': 181, 'after walker': 4, 'walker was': 244, 'was concerned': 250, 'concerned for': 46, 'for another': 76, 'another walker': 23, 'walker and': 240, 'and his': 13, 'his two': 102, 'two dogs': 234, 'dogs who': 60, 'who was': 270, 'was the': 257, 'the high': 205, 'high side': 94, 'side of': 189, 'of lingmell': 148, 'lingmell gill': 122, 'gill which': 83, 'which looked': 264, 'looked to': 127, 'to be': 221, 'be unpassable': 33, 'unpassable due': 236, 'due to': 65, 'to the': 231, 'the heavy': 204, 'heavy rainfall': 93, 'rainfall however': 171, 'however the': 105, 'the walker': 214, 'was able': 247, 'able to': 0, 'to communicate': 222, 'communicate his': 44, 'his intentions': 99, 'intentions to': 112, 'to re': 229, 're ascend': 172, 'ascend to': 24, 'to higher': 226, 'higher elevation': 95, 'elevation cross': 66, 'cross the': 53, 'the gill': 203, 'gill and': 82, 'and take': 20, 'take an': 196, '

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [18]:
cleaned_data = data.dropna(subset = ['main_text', 'Incident_Type','month','hrs','staff','start_time'])

In [21]:
variables = ['main_text', 'Incident_Type','month','hrs','staff','start_time']

cleaned_data = data.dropna(subset = variables)
cleaned_data.shape

(1208, 23)

In [25]:
X = cleaned_data[['month', 'hrs','staff','start_time']]
X.astype(
         {'hrs':'float'},
         {'staff':'int'},
         )

,month,hrs,staff,start_time
0,11,0.2,2,12:47
1,11,1.4,2,21:36
2,11,5.6,7,16:18
3,10,1.2,7,16:30
4,10,0.3,1,16:16
...,...,...,...,...
1405,1,6.0,19,10:00
1406,1,4.2,11,13:17
1407,1,4.2,13,16:47
1408,1,1.9,20,17:38


In [ ]:
X.start_time.dateT

In [19]:
cleaned_data

,title,date,Incident,Type,Location,Weather,Tagged,Incident_Type,Incident_Cause,url,...,end_time,hrs,staff,location,total_hrs,Other Agencies,Diagnosis,year,month,year_month
0,"Lingmell Gill Path, Scafell Pike – Tue 4th Nov...",2025-11-04,145 in 2025,Alert — Other,"Lingmell Gill Path, Scafell Pike","[Cold, Rain]",[False Alarm],Alert,Other,https://www.wmrt.org.uk/incidents/lingmell-gil...,...,13:00,0.2,2,NY195074,0.4,NaN,NaN,2025,11,2025-11-01
1,"Pikes Crag, Scafell Pike – Sun 2nd Nov 2025",2025-11-02,144 in 2025,Alert — Lost,"Pikes Crag, Scafell Pike","[Cloudy Summits, Cold, Mist]",[Lost],Alert,Lost,https://www.wmrt.org.uk/incidents/pikes-crag-s...,...,22:58,1.4,2,NY 210071,2.8,[Cumbria Police],NaN,2025,11,2025-11-01
2,"Broadcrag Tarn, Scafell Pike – Sun 2nd Nov 2025",2025-11-02,143 in 2025,Limited Callout — Lost,"Broadcrag Tarn, Scafell Pike","[Cloudy Summits, Cold, Mist]","[Lost, Party Separated]",Limited Callout,Lost,https://www.wmrt.org.uk/incidents/broad-crag-s...,...,21:53,5.6,7,NY213065,39,[Cumbria Police],NaN,2025,11,2025-11-01
3,"Green How, Scafell – Fri 31st Oct 2025",2025-10-31,142 in 2025,Limited Callout — Overdue,"Green How, Scafell","[Cold, Rain, Windy]","[Lost, Overdue]",Limited Callout,Overdue,https://www.wmrt.org.uk/incidents/green-how-sc...,...,17:41,1.2,7,NY 198066,8,[Cumbria Police],NaN,2025,10,2025-10-01
4,"Lingmell Col, Scafell Pike – Thu 30th Oct 2025",2025-10-30,141 in 2025,Alert — Lost,"Lingmell Col, Scafell Pike","[Cold, Drizzle, Mist]",[Lost],Alert,Lost,https://www.wmrt.org.uk/incidents/lingmell-col...,...,16:32,0.3,1,NY 216077,0.3,[Cumbria Police],NaN,2025,10,2025-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,Whitehaven – Fri 9th Jan 2015,2015-01-09,4 in 2015,Full Callout — Other,Whitehaven,NaN,NaN,Full Callout,Other,https://www.wmrt.org.uk/incidents/whitehaven-f...,...,16:00,6,19,NaN,114,[Cumbria Police],NaN,2015,1,2015-01-01
1406,Whitehaven – Wed 7th Jan 2015,2015-01-07,3 in 2015,Callout — Other,Whitehaven,NaN,NaN,Callout,Other,https://www.wmrt.org.uk/incidents/whitehaven-w...,...,17:30,4.2,11,NaN,46,[Duddon & Furness MRT],NaN,2015,1,2015-01-01
1407,Upper Eskdale – Tue 6th Jan 2015,2015-01-06,2 in 2015,Limited Callout — Lost,Upper Eskdale,NaN,NaN,Limited Callout,Lost,https://www.wmrt.org.uk/incidents/upper-eskdal...,...,21:00,4.2,13,NY 212 040,55,NaN,NaN,2015,1,2015-01-01
1408,Scafell Pike – Thu 1st Jan 2015,2015-01-01,1 in 2015,Full Callout — Overdue,Scafell Pike,NaN,NaN,Full Callout,Overdue,https://www.wmrt.org.uk/incidents/scafell-pike...,...,19:30,1.9,20,NaN,38,[Duddon & Furness MRT],NaN,2015,1,2015-01-01


In [20]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, y_train, 

In [17]:
data.columns

Index(['title', 'date', 'Incident', 'Type', 'Location', 'Weather', 'Tagged',
       'Incident_Type', 'Incident_Cause', 'url', 'main_text', 'rescue_info',
       'start_time', 'end_time', 'hrs', 'staff', 'location', 'total_hrs',
       'Other Agencies', 'Diagnosis', 'year', 'month', 'year_month'],
      dtype='object')